In [1]:
import torch

class CustomReLU(torch.nn.Module):
    """
    Custom implementation of the ReLU activation function.
    """
    def __init__(self):
        super(CustomReLU, self).__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Applies the ReLU function element-wise: max(0, x).
        Args:
            x (torch.Tensor): Input tensor
        Returns:
            torch.Tensor: Output tensor with ReALU applied
        """
        return torch.maximum(x, torch.zeros_like(x))

In [2]:
import torch

class ConvolutionalLayer:
    """
    A utility class to create a convolutional layer with a custom ReLU activation.
    """
    @staticmethod
    def create(in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int) -> torch.nn.Sequential:
        """
        Creates a convolutional layer with a custom ReLU activation.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
            kernel_size (int): Size of the convolution kernel
            stride (int): Stride of the convolution
            padding (int): Padding added to the input
        Returns:
            torch.nn.Sequential: A sequential layer containing Conv2d and CustomReLU(+Batch Normalization)
        """
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            torch.nn.BatchNorm2d(out_channels),
            CustomReLU()
        )

In [3]:
import torch

class FullyConnectedLayer:
    """
    A utility class to create Fully Connected (FC) layers with Xavier initialization.
    """
    @staticmethod
    def Dense(input_dim: int, output_dim: int) -> torch.nn.Linear:
        """
        Creates an FC layer and applies Xavier initialization.
        Args:
            input_dim (int): Number of input features
            output_dim (int): Number of output features
        Returns:
            torch.nn.Linear: Initialized Fully-Connected layer
        """
        layer = torch.nn.Linear(input_dim, output_dim, bias=True)
        torch.nn.init.xavier_uniform_(layer.weight)
        return layer

In [4]:
import torch

class InceptionV1(torch.nn.Module):
    """
    Implements GoogLeNet's Inception V1 layer
    """
    def __init__(self, in_channels: int, out_1x1conv: int, 
                 reduce_3x3conv:int, out_3x3conv: int,
                 reduce_5x5conv: int, out_5x5conv: int, out_pooling: int) -> torch.nn.Sequential:
        """
        Creates a Inception layer with a custom ReLU activation Function.
        Args:
            in_channels (int): Number of input channels
            out_1x1conv (int): Number of 1x1 Convolution output channels

        Returns:
            torch.nn.Sequential:
        """
        super(InceptionV1, self).__init__()

        self.branch1 = ConvolutionalLayer.create(in_channels, out_1x1conv, 1, 1, 0)

        self.branch2 = torch.nn.Sequential(
            ConvolutionalLayer.create(in_channels, reduce_3x3conv, 1, 1, 0),
            ConvolutionalLayer.create(reduce_3x3conv, out_3x3conv, 3, 1, 1)
        )

        self.branch3 = torch.nn.Sequential(
            ConvolutionalLayer.create(in_channels, reduce_5x5conv, 1, 1, 0),
            ConvolutionalLayer.create(reduce_5x5conv, out_5x5conv, 5, 1, 2)
        )

        self.branch4 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvolutionalLayer.create(in_channels, out_pooling, 1, 1, 0)
        )

    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], dim=1)

In [5]:
import torch

class CustomGoogLeNet(torch.nn.Module):
    """
    Implementation of the GoogLeNet model.
    Input: Image tensor (batch_size, 3, 224, 224)
    Output: Class scores (batch_size, 1000)
    """
    def __init__(self, dropout_rate=0.4):
        """
        Args:
            dropout_rate (float): Dropout Rate = 0.4(Base on Paper)
        """
        super(CustomGoogLeNet, self).__init__()

        # Convolutional and pooling layers
        self.layer1 = torch.nn.Sequential(
            ConvolutionalLayer.create(3, 64, 7, 2, 0),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.layer2 = torch.nn.Sequential(
            ConvolutionalLayer.create(64, 64, 1, 1, 0),
            ConvolutionalLayer.create(64, 192, 3, 1, 1),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.layer3 = torch.nn.Sequential(
            InceptionV1(192, 64, 96, 128, 16, 32, 32),
            InceptionV1(256, 128, 128, 192, 32, 96, 64),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.layer4 = torch.nn.Sequential(
            InceptionV1(480, 192, 96, 208, 16, 48, 64),
            InceptionV1(512, 160, 112, 224, 24, 64, 64),
            InceptionV1(512, 128, 128, 256, 24, 64, 64),
            InceptionV1(512, 112, 144, 288, 32, 64, 64),
            InceptionV1(528, 256, 160, 320, 32, 128, 128),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.layer5 = torch.nn.Sequential(
            InceptionV1(832, 256, 160, 320, 32, 128, 128),
            InceptionV1(832, 384, 192, 384, 48, 128, 128),
            torch.nn.AdaptiveAvgPool2d((1, 1))
        )

        # Fully Connected layers and dropout
        self.layer_drop = torch.nn.Dropout(p=dropout_rate)

        self.layer6 = FullyConnectedLayer.Dense(1 * 1 * 1024, 1000)

        self.layer7 = FullyConnectedLayer.Dense(1000, 1000)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines the forward pass of the model.
        Args:
            x (torch.Tensor): Input image tensor (batch_size, 3, 224, 224)
        Returns:
            torch.Tensor: Class scores (batch_size, 1000)
        """
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer_drop(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.layer6(x)
        x = self.layer7(x)
        return x

In [6]:
from torchsummary import summary

model = CustomGoogLeNet()
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 109, 109]           9,472
       BatchNorm2d-2         [-1, 64, 109, 109]             128
        CustomReLU-3         [-1, 64, 109, 109]               0
         MaxPool2d-4           [-1, 64, 55, 55]               0
            Conv2d-5           [-1, 64, 55, 55]           4,160
       BatchNorm2d-6           [-1, 64, 55, 55]             128
        CustomReLU-7           [-1, 64, 55, 55]               0
            Conv2d-8          [-1, 192, 55, 55]         110,784
       BatchNorm2d-9          [-1, 192, 55, 55]             384
       CustomReLU-10          [-1, 192, 55, 55]               0
        MaxPool2d-11          [-1, 192, 28, 28]               0
           Conv2d-12           [-1, 64, 28, 28]          12,352
      BatchNorm2d-13           [-1, 64, 28, 28]             128
       CustomReLU-14           [-1, 64,